In [43]:
%pip install -i https://pypi.gurobi.com gurobipy
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from random import randrange
import string
import math

Looking in indexes: https://pypi.gurobi.com, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
class shifts:
  def __init__(self, Fx = 1, Fy = 1, E = 50, S = 1, l = 20):
    self.Fx = Fx
    self.Fy = Fy
    self.S = S
    self.E = E    
    self.ES =  E * S
    self.l = l

  def u_x(self):
    return 4 * (self.Fx - 2 * self.Fy) * self.l / (self.ES)

  def u_y(self):
    return self.Fy * 25 * self.l / (self.ES) - 2 * self.u_x() + self.u_c()

  def u_c(self):
    return self.Fy * self.l / (self.ES)

  def inputs_print(self):
    print('Fx =', self.Fx, 'Fy =', self.Fy, 'E =', self.E, 'S =', self.S, 'l =', self.l)\

  def shift_print(self):
    self.inputs_print()
    self.numerical()

    print('analytic_result:')
    print('u_x(A) =', self.u_x())
    print('u_y(A) =', self.u_y())
    print('u_y(C) =', self.u_c())
    print("\n")


  def result_print(u_xa, u_ya, u_yc):
    print('gurobi_results:')
    print('u_x(A) =', u_xa)
    print('u_y(A) =', u_ya)
    print('u_y(C) =', u_yc)
    print('\n')

  def numerical(self):
    problem = gp.Model("first_problem")
    u = problem.addVars(range(3), lb = -GRB.INFINITY, ub = GRB.INFINITY, vtype=GRB.CONTINUOUS, name="u")

    obj = 0
    upd_l = []
    upd_l.append(2 * self.l - u[0])
    upd_l.append(5**0.5 * self.l + (u[2] - u[1] - 2 * u[0]) / (5**0.5))
    upd_l.append(self.l + u[2])

    obj += self.ES * self.l * ((2 * self.l - upd_l[0]) / (2 * self.l)) ** 2 / 2
    obj += self.ES * self.l * ((5**0.5 * self.l - upd_l[1]) / (5**0.5 * self.l)) ** 2 / 2
    obj += self.ES * self.l * ((self.l - upd_l[2]) / self.l) ** 2 / 2
    obj += -self.Fx * u[0] - self.Fy * u[1]

    problem.setObjective(obj, GRB.MINIMIZE)
    problem.update()
    problem.optimize()

    #self.result_print(u[0].x, u[1].x, u[2].x)
    print('gurobi_results:')
    print('u_x(A) =', u[0].x)
    print('u_y(A) =', u[1].x)
    print('u_y(C) =', u[2].x)
    print('\n')


In [45]:
shift = shifts()
shift.shift_print()
print()
shift = shifts(1, 2)
shift.shift_print()
print()
shift = shifts(1, 2, 70)
shift.shift_print()
shift = shifts(0, 0, 70)
shift.shift_print()

Fx = 1 Fy = 1 E = 50 S = 1 l = 20
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0x8c6ff768
Model has 6 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 0 rows, 3 columns, 0 nonzeros
Presolved model has 6 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 5
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   0.00000000e+00  0.00000000e+0